In [3]:
from flwr_datasets.partitioner import DirichletPartitioner

In [4]:
from flwr_datasets import FederatedDataset

In [2]:
from torch.utils.data import Subset

In [9]:
import torch

In [5]:
num_partitions = 4
alpha_dir = 0.1

In [6]:
partitioner = DirichletPartitioner(
            num_partitions=num_partitions,
            partition_by="label",
            alpha=alpha_dir,
            min_partition_size=0,
            self_balancing=False
        )

In [7]:
fds = FederatedDataset(
            dataset="mnist",
            partitioners={"train": partitioner}
        )

In [27]:
train_partition = fds.load_partition(0, split="train")

In [29]:
train_partition = train_partition.select(range(num_samples))

In [30]:
train_partition

Dataset({
    features: ['image', 'label'],
    num_rows: 1712
})

In [18]:
num_samples = int(len(train_partition)/10)
indices = list(range(num_samples))
train_partition = Subset(train_partition, indices)

In [19]:
train_partition

In [20]:
from torchvision.transforms import Compose, ToTensor, Normalize

In [31]:
pytorch_transforms = Compose([
    ToTensor(),
    Normalize((0.5,), (0.5,))
])

def apply_transforms(batch):
    batch["image"] = [pytorch_transforms(img) for img in batch["image"]]
    return batch

In [32]:
train_partition = train_partition.with_transform(apply_transforms)